# DCOPF

In [1]:
import scipy
import sympy as smp
import numpy as np

import andes
import ams

from ams.solver.pypower.makePTDF import makePTDF
from ams.solver.pypower.makeBdc import makeBdc

import pandapower as pp

from pandapower.pypower.makePTDF import makePTDF as ppmakePTDF
from pandapower.pd2ppc import _pd2ppc

import pandapower.networks as ppnet


In [2]:
andes.config_logger(stream_level=10)

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx']
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.0648 seconds.
Setting internal address for Bus
Setting internal address for PQ
Setting internal address for PV
Setting internal address for Slack
Setting internal address for Shunt
Setting internal address for Line
Setting internal address for Area
System internal structure set up in 0.0318 seconds.


In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              setup=True,)

ppc = ams.io.pypower.system2ppc(sp)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m']
Input format guessed as matpower.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0295 seconds.
System set up in 0.0009 seconds.


In [6]:
Bbus, Bf, ptdf = sp.mats.PTDF

Bbus:(14, 14)
/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:285: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  warn('spsolve is more efficient when sparse b '
/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)
Bf:(20, 14), dTheta:(14, 14)


In [7]:
ppn = ppnet.case14()

pp.rundcpp(ppn)

_, ppci = _pd2ppc(ppn)

ppptdf = ppmakePTDF(ppci["baseMVA"], ppci["bus"], ppci["branch"],)


In [16]:
ptdf.shape

(20, 14)

In [17]:
ppptdf.shape

(20, 14)

In [ ]:
pp.rundcpp(ppc)

ptdf = ppmakePTDF(ppc["baseMVA"], ppc["bus"], ppc["branch"],
                  )

In [ ]:
PTDF = makePTDF(ppc["baseMVA"], ppc["bus"], ppc["branch"])

In [ ]:
ppc = sp.